In [1]:
%load_ext autoreload
%autoreload 2
from prepare.preprocessing import process_all_scans
from prepare.patch_extraction import extract_negative_patches_from_candidates, extract_patches_from_annotations
from prepare.build_training_index import build_training_index, create_balanced_training_csv
from prepare.classes import LunaPatchDataset, Advanced3DAugment
from torch.utils.data import random_split, DataLoader

In [ ]:
LUNA_PATH = r"D:\archive"
OUTPUT_PATH = r"D:\output"
PREPROCESSED_OUTPUT = r"D:\output\preprocessed_luna16"
ANNOTATION_FILE = LUNA_PATH + r"\annotations.csv"
CANDIDATES_FILE = LUNA_PATH + r"\candidates_V2\candidates_V2.csv"
PATCH_OUTPUT = OUTPUT_PATH + r"\patches"
TRAINING_FILE = OUTPUT_PATH + r"\training_balanced.csv"
METADATA_FILE = PREPROCESSED_OUTPUT + r"\preprocessed_metadata.csv"

In [ ]:
# --- Stage 1: Preprocess All Scans ---
process_all_scans(LUNA_PATH, PREPROCESSED_OUTPUT)

In [ ]:
# --- Stage 2: Extract 3D Patches for Training ---
extract_patches_from_annotations(
    annotation_csv=ANNOTATION_FILE,
    metadata_csv=METADATA_FILE,
    output_folder=PATCH_OUTPUT,
    patch_size=32
)

In [ ]:
extract_negative_patches_from_candidates(
    candidates_csv=CANDIDATES_FILE,
    annotations_csv=ANNOTATION_FILE,
    metadata_csv=METADATA_FILE,
    output_folder=PATCH_OUTPUT,
    patch_size=32,
    max_negatives_per_scan=10
)

In [ ]:
# --- Stage 3: Build the Balanced Training Index CSV ---
create_balanced_training_csv(
    patch_folder=PATCH_OUTPUT,
    output_csv=TRAINING_FILE,
    oversample_pos=True,
    downsample_neg=True
)

In [ ]:
# debug for each tensor should ne equal size error
import torch
for i in range(1200):
    x, y = train_dataset[i]
    if x.shape != torch.Size([1, 32, 32, 32]):
        print(f"Sample {i} shape: {x.shape}")